# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [2]:
from google.colab import drive
drive.mount('/content/drive')
with directory("/content/drive/My Drive/duckietown_dataset"):
  print(os.getcwd())
  print(os.listdir())

  with open("./test.txt", "w") as f:  # please verify that you did get a new file in your Drive
    f.write("Hello, world!")
os.chdir('/content/drive/My Drive/duckietown_dataset')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/duckietown_dataset
['images', 'frames', 'labels', 'annotation', 'test.txt', 'duckietown_dataset']


In [3]:

if not os.path.exists("SENTINEL"):
  prun("mkdir duckietown_dataset")
  prun("mv train duckietown_dataset && mv val duckietown_dataset")

mkdir: cannot create directory ‘duckietown_dataset’: File exists

mv: cannot stat 'train': No such file or directory



# Next, we will clone Yolov5 into your google drive. 

In [4]:
!git clone https://github.com/duckietown/yolov5.git -b dt-obj-det
!cd yolov5 && pip3 install -r requirements.txt
if not os.path.exists("SENTINEL"):
  run("mv duckietown_dataset yolov5")
!touch SENTINEL

Cloning into 'yolov5'...
remote: Enumerating objects: 6154, done.
remote: Total 6154 (delta 0), reused 0 (delta 0), pack-reused 6154
Receiving objects: 100% (6154/6154), 8.45 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (4219/4219), done.
     |████████████████████████████████| 645kB 6.9MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# And we're ready to train! This step will take a hour or so.

Notice that we're only training for 10 epochs. That's probably not enough!

In [5]:
!mv yolov5/best.pt yolov5/best_old.pt
!cd yolov5 && pip3 install -r requirements.txt && python3 train.py --img 416 --batch 16 --epochs 10 --data duckietown.yaml --weights yolov5s.pt

import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")

mv: cannot stat 'yolov5/best.pt': No such file or directory
github: up to date with https://github.com/duckietown/yolov5 ✅
YOLOv5 🚀 2021-6-21 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/duckietown.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-21 03:05:21.483325: I tensorflow/stream_executor/pl

# Next, we can upload your model to Duckietown's cloud!

In [6]:
!pip3 install git+https://github.com/duckietown/lib-dt-mooc-2021

  Cloning https://github.com/duckietown/lib-dt-mooc-2021 to /tmp/pip-req-build-mmx6c4lv
  Running command git clone -q https://github.com/duckietown/lib-dt-mooc-2021 /tmp/pip-req-build-mmx6c4lv
     |████████████████████████████████| 122kB 8.7MB/s 
  Created wheel for dt-mooc-2021: filename=dt_mooc_2021-0.0.0-cp37-none-any.whl size=6449 sha256=02bc6a7d58d06804f0d43e42fe5d634c76686d714f85098cef89d35a36fbb594
  Stored in directory: /tmp/pip-ephem-wheel-cache-vmv2bodc/wheels/4d/a5/d1/d0b4812d9a01d0ca35a6eac789af1ada336277ab57e746e3f9
  Created wheel for dt-data-api-daffy: filename=dt_data_api_daffy-0.1.10-cp37-none-any.whl size=13253 sha256=36c2f2b90642bc6b86ce1c8b8fde1365c9ecb39475215453896e00e4eb9a7896
  Stored in directory: /root/.cache/pip/wheels/fa/93/a2/146d2179d3d51039049bcaa9bd834991144bf1f075829fa2a5
  Created wheel for dt-authentication-daffy: filename=dt_authentication_daffy-0.1.5-cp37-none-any.whl size=3419 sha256=5aaeb7de3e832a866d684a750dbf29b60829f8ed623f91b36e1fb0cd214db9c

In [9]:
from dt_mooc.cloud import Storage
import sys
import torch

def select_device(device='', batch_size=None):
    import torch
    # device = 'cpu' or '0' or '0,1,2,3'
    cpu = device.lower() == 'cpu'
    if cpu:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # force torch.cuda.is_available() = False
    elif device:  # non-cpu device requested
        os.environ['CUDA_VISIBLE_DEVICES'] = device  # set environment variable
        assert torch.cuda.is_available(), f'CUDA unavailable, invalid device {device} requested'  # check availability

    cuda = not cpu and torch.cuda.is_available()

    return torch.device('cuda:0' if cuda else 'cpu')

sys.path.insert(0, './yolov5')
model = torch.load("./yolov5/best.pt", map_location=select_device("cpu"))['model'].float()  # load to FP32
model.to(select_device("cpu")).eval()

storage = Storage("dt1-3nT8KSoxVh4Mf9otux6HBhr9jvbAEevKFNfz14CY82XS6Ep-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfWsn4WGvasSc3ojMeADVE4jckqCvnaPVB7")

storage.upload_yolov5("yolov5", model, "./yolov5/best.pt")

Uploading file `best.pt`...



File `best.pt` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.pt`.
Uploading file `best.pt.wts`...

File `best.pt.wts` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.wts`.
Uploading file `best.pt.wts.sha256`...

File `best.pt.wts.sha256` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.sha256`.


# Done!

We're done training! You can go back to the `Training` notebook